In [1]:
import re, string,os
from glob import glob as gb
import pandas as pd
from tqdm import tqdm
from collections import Counter
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from functions import *
import calendar
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from nltk.corpus import stopwords

base_path = "/home/ruben/Documents/GitHub/ParlaMintCase"

In [2]:
tf = pd.read_csv(base_path + '/resources/keywords-expertise-translation.csv')
tf_science = {language:dict(zip(tf[tf['language'] == language]['word'],tf[tf['language'] == language]['translation'])) for language in list(set(tf['language']))}
tf_science = {k:{x:i for x,i in v.items() if x not in ["policy","program","measures","knowledge","information"]} for k,v in tf_science.items()}
tf_science.update({"gb":{k:k for k,v in tf_science['nl'].items()}})

In [5]:
df = data_loader.period(language="gb",data_version="lemmatized",start_date="2019-08",end_date="2020-11")
# df = df[df.id.astype(str).str.contains('-commons')]
df['date'] = [utils.find_date(x)[:7] for x in df['id']]
df = utils.add_metadata(df,'gb')
df['text'] = utils.preprocess_(df['text'],stopwords=set(stopwords.words('english')) | set("hon. friend right gentleman".split(' ')))
# df['text'] = utils.preprocess_(df['text'],stopwords=set(stopwords.words('dutch')) | set("wij zij het jij ons jullie".split(' ')))
df['text'] = [re.sub(r'[^A-Za-z0-9 ]+', '', str(s)) for s in df['text']]

looking for data in  /media/ruben/Elements/ParlaMint/gb/gb-ana-txt/
found 317 files


In [8]:
r = pd.DataFrame()
for m in utils.month_generator("2019-12","2020-06"):
    dm = df[df['date'] == m]
    dm['l'] = ["science" if any(n in t for n in tf_science['gb'].values()) == True else "other" for t in dm['text']]
    
    dm2 = dm[dm['l'] == 'science']
    dm2 = dm2.append(dm[dm['l'] != 'science'].reset_index(drop=True).sample(300)).reset_index(drop=True)
    tfidf_obj,docterms = tfidf.get_docterms(dm2,'text')
    tftw = tfidf.get_topterms(tfidf_obj,docterms,dm2,'l')
    r[m] = tftw['science'][:15]

In [9]:
r

,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10
0,wetenschappelijk,wetenschap,wetenschap,stellen,wetenschappelijk,mens,expert,expert,expert,wetenschap,onderzoek,welke,expert,wetenschapper
1,vragen,wetenschappelijk,onderzoek,onderzoek,mens,wetenschappelijk,maatregel,kabinet,minister,motie,vragen,expert,kabinet,kabinet
2,echt,nederland,wetenschappelijk,wetenschappelijk,wetenschap,wetenschap,kabinet,app,onderzoek,wetenschappelijk,weten,ggd,eigenlijk,maatregel
3,laten,stellen,kabinet,onderwijs,wet,kijken,virus,betrekken,echt,jaar,minister,advies,advies,wetenschappelijk
4,jaar,kabinet,onderwijs,jaar,onderzoek,vragen,weten,weer,waar,kijken,stellen,voldoen,wetenschappelijk,wetenschap
5,kamer,nemen,eerste,expert,nieuw,echt,wetenschappelijk,vragen,denken,commissie,jaar,motie,natuurlijk,wet
6,onderzoek,kamer,nodig,wetenschap,vragen,zien,land,nodig,kabinet,weten,kamer,inderdaad,staan,weer
7,zitten,rivm,punt,weten,manier,waar,advies,expertise,kijken,krijgen,denken,zorgen,rivm,onderwijs
8,nodig,onderzoek,alleen,nederland,alleen,zitten,moment,motie,wetenschappelijk,zorgen,collega,verspreiding,nou,gewoon
9,wetenschapper,aantal,juist,groot,stellen,laten,nodig,gewoon,risico,natuurlijk,krijgen,weten,aantal,aantal


In [8]:
r.to_csv(os.path.join(base_path,"results/gb-topterms-tfidf.csv"),index=False)

In [61]:
cv=CountVectorizer(max_df=0.85,stop_words=list(set(stopwords.words('english'))),max_features=10000)
word_count_vector=cv.fit_transform(list(d['text']))
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

feature_names=cv.get_feature_names()

In [62]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):

    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [64]:
d['month'] = [x[:7] for x in d['date']]

In [66]:
for da in sorted(list(set(d.date))):
    doc = " ".join(d[d['date'] == da]['text'])
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    print(da,'\t'," ".join(extract_topn_from_vector(feature_names,sorted_items,10).keys()))

2020-01-07 	 eu clause citizen government make right status iran would people
2020-01-08 	 ireland northern clause european eu government directive amendment union agreement
2020-01-09 	 hon member european government house eu brexit deal union people
2020-01-13 	 world foreign iran hon member government international friend global country
2020-01-14 	 school government education speech year need make iran local people
2020-01-15 	 climate government need green speech change year carbon make revolution
2020-01-16 	 care nhs health people government hospital need service social member
2020-01-20 	 government work people year need speech building school make economy
2020-01-21 	 fire farmer building grenfell payment bill government cladding year make
2020-01-22 	 government school broadband hon make meal child minister bill say
2020-01-23 	 holocaust jewish antisemitism people auschwitz memorial camp member debate hon
2020-01-27 	 nhs health hospital people year hon funding need governme

{'pm': 0.258,
 'house': 0.243,
 'ballot': 0.192,
 'australian': 0.182,
 'wish': 0.162,
 'open': 0.157,
 'thursday': 0.155,
 'room': 0.152,
 'australia': 0.149,
 'book': 0.145}